In [1]:
from pathlib import Path
import xarray as xr
import pandas as pd
import numpy as np
#from scripts.plot_utils import map_it
import matplotlib.pyplot as pp

In [2]:
project_path = Path.cwd()
data_path = Path.home() / 'data' / 'craig_pfc_2023'
file = 'pcc-sims-202112.nc'
file_path = data_path / file
assert file_path.exists()

In [3]:
ds = xr.open_dataset(file_path, decode_cf=False)  # Or from siphon.ncss

In [4]:
ds

<xarray.Dataset> Size: 4GB
Dimensions:     (date: 31, lon: 288, lat: 234, component: 6, wavelength: 501)
Coordinates:
  * date        (date) int64 248B 0 1 2 3 4 5 6 7 8 ... 23 24 25 26 27 28 29 30
  * lon         (lon) float64 2kB -180.0 -178.7 -177.5 ... 177.5 178.7 180.0
  * lat         (lat) float64 2kB -84.0 -83.33 -82.67 -82.0 ... 70.07 70.73 71.4
  * component   (component) <U3 72B 'dia' 'chl' 'cya' 'coc' 'din' 'pha'
  * wavelength  (wavelength) int64 4kB 250 251 252 253 254 ... 747 748 749 750
Data variables: (12/21)
    tot         (date, lon, lat) float32 8MB ...
    dtc         (date, lon, lat) float32 8MB ...
    pic         (date, lon, lat) float32 8MB ...
    cdc         (date, lon, lat) float32 8MB ...
    t           (date, lon, lat) float32 8MB ...
    s           (date, lon, lat) float32 8MB ...
    ...          ...
    rnh         (date, lon, lat) float32 8MB ...
    rno         (date, lon, lat) float32 8MB ...
    sil         (date, lon, lat) float32 8MB ...
    zoo         (date, lon, lat) float32 8MB ...
    phy         (date, lon, lat, component) float32 50MB ...
    rrs         (date, lon, lat, wavelength) float32 4GB ...

In [6]:
ds.lon.min(), ds.lon.max()

(<xarray.DataArray 'lon' ()> Size: 8B
 array(-180.),
 <xarray.DataArray 'lon' ()> Size: 8B
 array(180.))

In [5]:
ds.lat.min(), ds.lat.max()

(<xarray.DataArray 'lat' ()> Size: 8B
 array(-84.),
 <xarray.DataArray 'lat' ()> Size: 8B
 array(71.4))

In [5]:
vars_list = [k for k in ds.data_vars.keys()]

In [ ]:
ds.date.min(), ds.date.max()


(<xarray.DataArray 'date' ()> Size: 8B
 array(0),
 <xarray.DataArray 'date' ()> Size: 8B
 array(30))

In [14]:
phyto_components = ds.component.to_numpy().tolist()
rrs_λ = ds.wavelength.to_numpy().tolist()

n_lat = ds.dims.get('lat')
n_date = ds.dims.get('date')
n_lon = ds.dims.get('lon')
n_wave = len(rrs_λ)
n_comp = len(phyto_components)
datlonlat = n_date * n_lon * n_lat

/var/folders/c9/p_x94m2j357984db558g2n9m0000gn/T/ipykernel_51524/2692960338.py:4: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  n_lat = ds.dims.get('lat')
<frozen _collections_abc>:807: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
/var/folders/c9/p_x94m2j357984db558g2n9m0000gn/T/ipykernel_51524/2692960338.py:5: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  n_date = ds.dims.get('date')
/var/fo

### Extract to DataFrames

In [15]:
vars_list

['tot',
 'dtc',
 'pic',
 'cdc',
 't',
 's',
 'alk',
 'dic',
 'doc',
 'fco',
 'h',
 'irn',
 'pco',
 'tpp',
 'pp',
 'rnh',
 'rno',
 'sil',
 'zoo',
 'phy',
 'rrs']

In [19]:
datlonlat = n_date * n_lon * n_lat

tot_cphyl = ds.tot.to_numpy().reshape(datlonlat, 1)
dtc = ds.dtc.to_numpy().reshape(datlonlat, 1)
pic = ds.pic.to_numpy().reshape(datlonlat, 1)
cdc = ds.cdc.to_numpy().reshape(datlonlat, 1)
temp = ds.t.to_numpy().reshape(datlonlat, 1)
sal = ds.s.to_numpy().reshape(datlonlat, 1)
alk = ds.alk.to_numpy().reshape(datlonlat, 1)
dic = ds.dic.to_numpy().reshape(datlonlat, 1)
doc = ds.doc.to_numpy().reshape(datlonlat, 1)
fco = ds.fco.to_numpy().reshape(datlonlat, 1)
h = ds.h.to_numpy().reshape(datlonlat, 1)
iron = ds.irn.to_numpy().reshape(datlonlat, 1)
pco = ds.pco.to_numpy().reshape(datlonlat, 1)
tpp = ds.tpp.to_numpy().reshape(datlonlat, 1)
pp = ds.pp.to_numpy().reshape(datlonlat, n_comp)
rnh = ds.rnh.to_numpy().reshape(datlonlat, 1)
nitrate = ds.rno.to_numpy().reshape(datlonlat, 1)
sil = ds.sil.to_numpy().reshape(datlonlat, 1)
zoo = ds.zoo.to_numpy().reshape(datlonlat, 1)
rrs = ds.rrs.to_numpy().reshape(datlonlat, n_wave)
phy = ds.phy.to_numpy().reshape(datlonlat, n_comp)
lat = np.repeat(ds.lat.to_numpy(), n_date * n_lon)

In [29]:
np.nanmin(h), np.nanmax(h)

(np.float32(4.9999995), np.float32(200.00002))

In [20]:
df_rrs = pd.DataFrame(rrs, columns=rrs_λ)
df_phy = pd.DataFrame(np.c_[phy, tot_cphyl], columns=phyto_components+['tot_cphyl'])
df_env = pd.DataFrame(
    np.c_[lat, temp, nitrate, iron, sil], columns=['lat', 'temp', 'nitrate', 'iron', 'silica'])
df_env2 = pd.DataFrame(
    np.c_[dtc, pic, cdc, alk, dic, doc, fco, pco, rnh, zoo], 
    columns=['dtc', 'pic', 'cdc', 'alk', 'dic', 'doc', 'fco', 'pco', 'rnh', 'zoo'])
df_pp = pd.DataFrame(np.c_[pp, tpp], columns=phyto_components+['tot_pp'])

In [25]:
df_phy.describe()

,dia,chl,cya,coc,din,pha,tot_cphyl
count,1.261607e+06,1.261607e+06,1.261607e+06,1.261607e+06,1.261607e+06,1.261607e+06,1.261607e+06
mean,6.999361e-02,2.271937e-02,1.916263e-02,3.387509e-02,6.067622e-04,1.201726e-01,2.665299e-01
std,1.264303e-01,4.888777e-02,2.569767e-02,7.426477e-02,6.901991e-03,4.219097e-01,4.068106e-01
min,0.000000e+00,0.000000e+00,2.291250e-33,0.000000e+00,2.566806e-33,6.308696e-37,3.129146e-13
25%,2.151143e-04,9.006618e-14,2.361023e-13,3.536130e-05,4.563332e-23,1.560003e-32,9.044519e-02
50%,5.010420e-03,9.568140e-05,2.593337e-03,3.831222e-03,2.671561e-17,4.426940e-19,1.705623e-01
75%,8.362008e-02,1.864595e-02,3.433050e-02,2.870553e-02,1.785598e-10,7.129596e-04,3.183219e-01
max,1.809482e+00,1.114668e+00,4.471208e-01,1.650084e+00,7.721317e-01,1.186463e+01,1.186463e+01


In [36]:
df_all = pd.concat((df_phy, df_rrs, df_env), axis=1)

In [38]:
df_all.to_parquet(data_path / 'step_1_extracted' / 'df_all.pqt')

In [39]:
df_rrs.to_parquet(data_path / 'step_1_extracted' / 'df_rrs.pqt')

In [42]:
df_phy.to_parquet(data_path / 'step_1_extracted' / 'df_phy.pqt')

In [43]:
df_env.to_parquet(data_path / 'step_1_extracted' / 'df_env.pqt')

In [35]:
df_rrs.shape, df_env.shape, df_phy.shape

((2089152, 501), (2089152, 5), (2089152, 7))

In [48]:
df_env.head()

,lat,temp,nitrate,iron,silica
0,-84.0,NaN,NaN,NaN,NaN
1,-84.0,NaN,NaN,NaN,NaN
2,-84.0,NaN,NaN,NaN,NaN
3,-84.0,NaN,NaN,NaN,NaN
4,-84.0,NaN,NaN,NaN,NaN


In [49]:
df_env.describe()

,lat,temp,nitrate,iron,silica
count,2.089152e+06,1.261607e+06,1.261607e+06,1.261607e+06,1.261607e+06
mean,-6.300000e+00,1.616145e+01,5.515115e+00,3.142371e-01,7.914522e+00
std,4.505225e+01,1.032092e+01,7.313258e+00,2.591170e-01,1.001315e+01
min,-8.400000e+01,-1.017889e+00,8.000412e-04,5.048735e-03,1.938913e-03
25%,-4.531674e+01,5.567214e+00,4.595803e-02,1.703310e-01,1.419863e+00
50%,-6.300000e+00,1.889113e+01,4.257810e-01,2.184942e-01,3.390273e+00
75%,3.271674e+01,2.593674e+01,1.100956e+01,3.516443e-01,9.888976e+00
max,7.140000e+01,3.051188e+01,4.287701e+01,1.053397e+01,9.514703e+01


In [46]:
df_rrs.head()

,250,251,252,253,254,255,256,257,258,259,...,741,742,743,744,745,746,747,748,749,750
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
df_rrs.describe()

,250,251,252,253,254,255,256,257,258,259,...,741,742,743,744,745,746,747,748,749,750
count,1.261607e+06,1.261607e+06,1.261607e+06,1.261607e+06,1.261607e+06,1.261607e+06,1.261607e+06,1.261607e+06,1.261607e+06,1.261607e+06,...,1.261607e+06,1.261607e+06,1.261607e+06,1.261607e+06,1.261607e+06,1.261607e+06,1.261607e+06,1.261607e+06,1.261607e+06,1.261607e+06
mean,2.753939e-03,2.760335e-03,2.765184e-03,2.768637e-03,2.770863e-03,2.772059e-03,2.772450e-03,2.772271e-03,2.771795e-03,2.771315e-03,...,1.749838e-05,1.705293e-05,1.660858e-05,1.616145e-05,1.568660e-05,1.519160e-05,1.472462e-05,1.436814e-05,1.406985e-05,1.384782e-05
std,3.575387e-04,3.606649e-04,3.633946e-04,3.657585e-04,3.677939e-04,3.695456e-04,3.710662e-04,3.724144e-04,3.736561e-04,3.748642e-04,...,1.072021e-05,1.072566e-05,1.082674e-05,1.097574e-05,1.125343e-05,1.152935e-05,1.183014e-05,1.207732e-05,1.230047e-05,1.250261e-05
min,1.799369e-04,1.789097e-04,1.777569e-04,1.772388e-04,1.766177e-04,1.750987e-04,1.766700e-04,1.741120e-04,1.738909e-04,1.744357e-04,...,-1.972658e-05,-2.080763e-05,-2.096269e-05,-2.108125e-05,-2.115585e-05,-2.127285e-05,-2.137326e-05,-2.151000e-05,-2.168505e-05,-2.190489e-05
25%,2.559879e-03,2.564074e-03,2.567008e-03,2.568747e-03,2.569485e-03,2.569399e-03,2.568632e-03,2.567392e-03,2.565903e-03,2.564441e-03,...,1.167272e-05,1.164908e-05,1.155655e-05,1.137910e-05,1.112631e-05,1.083979e-05,1.058169e-05,1.036153e-05,1.017695e-05,1.002432e-05
50%,2.829355e-03,2.836162e-03,2.841324e-03,2.845025e-03,2.847448e-03,2.848825e-03,2.849315e-03,2.849214e-03,2.848840e-03,2.848463e-03,...,1.445736e-05,1.447497e-05,1.454502e-05,1.465000e-05,1.478719e-05,1.493718e-05,1.510769e-05,1.531821e-05,1.549135e-05,1.565062e-05
75%,3.021414e-03,3.030322e-03,3.037375e-03,3.042743e-03,3.046630e-03,3.049245e-03,3.050874e-03,3.051836e-03,3.052408e-03,3.052947e-03,...,2.472159e-05,2.425807e-05,2.370439e-05,2.311912e-05,2.254916e-05,2.186491e-05,2.120094e-05,2.061843e-05,2.006010e-05,1.985146e-05
max,4.109053e-03,4.144377e-03,4.178054e-03,4.210304e-03,4.241395e-03,4.271444e-03,4.300738e-03,4.329689e-03,4.358500e-03,4.387610e-03,...,2.762896e-04,2.732710e-04,2.714210e-04,2.689818e-04,2.683017e-04,2.676200e-04,2.663441e-04,2.656521e-04,2.655452e-04,2.642434e-04


In [44]:
df_phy.head()

,dia,chl,cya,coc,din,pha,tot_cphyl
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
df_phy.describe()

,dia,chl,cya,coc,din,pha,tot_cphyl
count,1.261607e+06,1.261607e+06,1.261607e+06,1.261607e+06,1.261607e+06,1.261607e+06,1.261607e+06
mean,6.999361e-02,2.271937e-02,1.916263e-02,3.387509e-02,6.067622e-04,1.201726e-01,2.665299e-01
std,1.264303e-01,4.888777e-02,2.569767e-02,7.426477e-02,6.901991e-03,4.219097e-01,4.068106e-01
min,0.000000e+00,0.000000e+00,2.291250e-33,0.000000e+00,2.566806e-33,6.308696e-37,3.129146e-13
25%,2.151143e-04,9.006618e-14,2.361023e-13,3.536130e-05,4.563332e-23,1.560003e-32,9.044519e-02
50%,5.010420e-03,9.568140e-05,2.593337e-03,3.831222e-03,2.671561e-17,4.426940e-19,1.705623e-01
75%,8.362008e-02,1.864595e-02,3.433050e-02,2.870553e-02,1.785598e-10,7.129596e-04,3.183219e-01
max,1.809482e+00,1.114668e+00,4.471208e-01,1.650084e+00,7.721317e-01,1.186463e+01,1.186463e+01


### Clean Data

In [59]:
df_all_clean = df_all.dropna()

In [63]:
valid_indices = df_all_clean.index

In [64]:
df_phy_clean = df_phy.loc[valid_indices]
df_env_clean = df_env.loc[valid_indices]
df_rrs_clean = df_rrs.loc[valid_indices]

In [67]:
df_env_clean.head()

,lat,temp,nitrate,iron,silica
9,-84.0,0.857763,12.907951,0.182768,28.479807
10,-84.0,0.846447,15.581422,0.282881,28.604376
11,-84.0,0.827969,16.857571,0.318571,29.003593
12,-84.0,0.810619,17.345680,0.326929,29.388382
13,-84.0,0.801506,17.232298,0.319819,29.552345


In [66]:
df_env.shape

(2089152, 5)

In [68]:
df_env_clean.describe()

,lat,temp,nitrate,iron,silica
count,1.261607e+06,1.261607e+06,1.261607e+06,1.261607e+06,1.261607e+06
mean,-6.470441e+00,1.616145e+01,5.515115e+00,3.142371e-01,7.914522e+00
std,4.505472e+01,1.032092e+01,7.313258e+00,2.591170e-01,1.001315e+01
min,-8.400000e+01,-1.017889e+00,8.000412e-04,5.048735e-03,1.938913e-03
25%,-4.531674e+01,5.567214e+00,4.595803e-02,1.703310e-01,1.419863e+00
50%,-6.633476e+00,1.889113e+01,4.257810e-01,2.184942e-01,3.390273e+00
75%,3.271674e+01,2.593674e+01,1.100956e+01,3.516443e-01,9.888976e+00
max,7.140000e+01,3.051188e+01,4.287701e+01,1.053397e+01,9.514703e+01


In [72]:
df_phy_clean.describe()

,dia,chl,cya,coc,din,pha,tot_cphyl
count,1.261607e+06,1.261607e+06,1.261607e+06,1.261607e+06,1.261607e+06,1.261607e+06,1.261607e+06
mean,6.999361e-02,2.271938e-02,1.916264e-02,3.387509e-02,6.067622e-04,1.201726e-01,2.665301e-01
std,1.264303e-01,4.888777e-02,2.569767e-02,7.426477e-02,6.901991e-03,4.219097e-01,4.068106e-01
min,0.000000e+00,0.000000e+00,2.291250e-33,0.000000e+00,2.566806e-33,6.308696e-37,3.129146e-13
25%,2.151143e-04,9.006618e-14,2.361023e-13,3.536130e-05,4.563332e-23,1.560003e-32,9.044519e-02
50%,5.010420e-03,9.568140e-05,2.593337e-03,3.831222e-03,2.671561e-17,4.426940e-19,1.705623e-01
75%,8.362008e-02,1.864595e-02,3.433050e-02,2.870553e-02,1.785598e-10,7.129596e-04,3.183219e-01
max,1.809482e+00,1.114668e+00,4.471208e-01,1.650084e+00,7.721317e-01,1.186463e+01,1.186463e+01


In [75]:
def clean_rrs_data(df):
    small_positive = 1e-20
    for col in df.columns:
        df[col] = np.where(df[col] < 0, small_positive, df[col])
    return df

In [81]:
df_rrs_clean.shape, df_rrs_clean.min().min()

((1261607, 501), 0.0)

In [84]:
df_all_clean.to_parquet(data_path / 'step_2_cleaned' / 'df_all.pqt')
df_rrs_clean.to_parquet(data_path / 'step_2_cleaned' / 'df_rrs.pqt')
df_env_clean.to_parquet(data_path / 'step_2_cleaned' / 'df_env.pqt')
df_phy_clean.to_parquet(data_path / 'step_2_cleaned' / 'df_phy.pqt')

/Users/erdemkarakoylu/mambaforge/envs/pymc_base/lib/python3.11/site-packages/pandas/io/parquet.py:189: UserWarning: The DataFrame has column names of mixed type. They will be converted to strings and not roundtrip correctly.
  table = self.api.Table.from_pandas(df, **from_pandas_kwargs)
